In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk import stem
stemmer = stem.PorterStemmer()
from nltk import word_tokenize
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
import string
punct = list(string.punctuation)
from collections import Counter
import requests
import pandas as pd
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
import numpy as np
import praw
import datetime
import json
import re
import emoji

In [2]:
reddit = praw.Reddit(user_agent='VAD',
                     client_id='Que1ZJxBJmaWI7A4VvmZKw', client_secret="AhSItN-CRonRy8kjdeTf3PKUXtyZwg",
                     username='photosynthesising', password='Addie12345')

# Conservative Reddit

In [3]:
# reddit post 1
# post ID
post = reddit.submission(id="jpu2i4")

#empty List
conred = []

# collect data
def get_comments(comment, comments_list):
    
    if isinstance(comment, praw.models.Comment):
        text = comment.body.replace("\n", " ")
        comments_list.append(text)

        for reply in comment.replies:
            get_comments(reply, comments_list)
    
    elif isinstance(comment, praw.models.MoreComments):
        for more_comments in comment.comments():
            get_comments(more_comments, comments_list)

# all comments and replies into empty list
for comment in post.comments:
    if len(conred) < 3000:
        get_comments(comment, conred)

In [4]:
conred

['When my buddy was done voting, a girl with the newspaper was interviewing people as they came out. She asked who he voted for, he said Trump, and he told her why. She then asked "what will you do if you wake up tomorrow morning and Biden is the new president?" He says, "get up and go to work."',
 'That’s the definition of being a republican. Life goes on.',
 'Nah it’s just the definition of being a normal citizen. But yes, it’s also the definition of being a good Republican. It’s the bad cookies that we hope everyone will denounce.',
 'That would be really nice.   One thing I have noticed, in both this sub and r/politics- a couple things we seem to all agree on is that companies like Facebook tried to influence voters. If you go to liberal subs, they were advocating for Trump. If you fi to conservative subs, it was Trump attack ads.   Second, is that no one really wants anyone to spend money on r didn\'t for really any purpose. Just look at the heavily awarded "AP calls race for Bide

In [5]:
#count number of responses
num_conred = len(conred)

print(num_conred)

3492


In [6]:
#check type of data in the lists
type(conred[0])

str

In [7]:
#remove all emojis
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

conred1 = []
for string in conred:
    conredem = emoji_pattern.sub('', string)
    conred1.append(conredem)

In [8]:
# remove any deleted or removed comments
for i, j in enumerate(conred1):
    if j == '''[removed]''' or j == '''[deleted]''':
        conred1.pop(i)

In [9]:
len(conred1)

3305

In [10]:
#check type of data in the lists
type(conred[0])

str

In [11]:
#tokenize
conred2 = []
for i in conred1:
    words = word_tokenize(i)
    conred2.append(words)

In [12]:
# remove all the punctuation
conred3 = conred2

for i in conred3:
    for index_j, j in enumerate(i):
        if j in punct:
            i.pop(index_j)

In [13]:
#lemmatize
conred4 = []

for i in conred3: 
    conred4.append([lemmatizer.lemmatize(j) for j in i])

In [14]:
#lower all cases
conred5 = [[j.lower() for j in i] for i in conred4]

In [15]:
#export to dataframe
big_df = pd.DataFrame()

big_df['comments'] = conred1
big_df['tokenized'] = conred2
big_df['lemmatized'] = conred4
big_df['lower'] = conred5

In [16]:
big_df

,comments,tokenized,lemmatized,lower
0,"When my buddy was done voting, a girl with the...","[When, my, buddy, was, done, voting, a, girl, ...","[When, my, buddy, wa, done, voting, a, girl, w...","[when, my, buddy, wa, done, voting, a, girl, w..."
1,That’s the definition of being a republican. L...,"[That, ’, s, the, definition, of, being, a, re...","[That, ’, s, the, definition, of, being, a, re...","[that, ’, s, the, definition, of, being, a, re..."
2,Nah it’s just the definition of being a normal...,"[Nah, it, ’, s, just, the, definition, of, bei...","[Nah, it, ’, s, just, the, definition, of, bei...","[nah, it, ’, s, just, the, definition, of, bei..."
3,That would be really nice. One thing I have ...,"[That, would, be, really, nice, One, thing, I,...","[That, would, be, really, nice, One, thing, I,...","[that, would, be, really, nice, one, thing, i,..."
4,"Oh i agree, but the right currently is being p...","[Oh, i, agree, but, the, right, currently, is,...","[Oh, i, agree, but, the, right, currently, is,...","[oh, i, agree, but, the, right, currently, is,..."
...,...,...,...,...
3300,Its not going to change,"[Its, not, going, to, change]","[Its, not, going, to, change]","[its, not, going, to, change]"
3301,They just did.,"[They, just, did]","[They, just, did]","[they, just, did]"
3302,Yea the people did that for them,"[Yea, the, people, did, that, for, them]","[Yea, the, people, did, that, for, them]","[yea, the, people, did, that, for, them]"
3303,"Doesn't matter, the Gap is wide enough in Neva...","[Does, n't, matter, the, Gap, is, wide, enough...","[Does, n't, matter, the, Gap, is, wide, enough...","[does, n't, matter, the, gap, is, wide, enough..."


In [17]:
#export to csv
big_df.to_excel('consreddit.xlsx')

# Republican Reddit

In [18]:
# reddit post 2
# post ID
post = reddit.submission(id="jnluu6")
                
#empty List
repred = []

# all comments and replies into empty list
for comment in post.comments:
    if len(repred) < 2600:  # set a limit here
        get_comments(comment, repred)
        
# collect data
def get_comments(comment, comments_list):
    
    if isinstance(comment, praw.models.Comment):
        text = comment.body.replace("\n", " ")
        comments_list.append(text)

        for reply in comment.replies:
            get_comments(reply, comments_list)
    
    elif isinstance(comment, praw.models.MoreComments):
        for more_comments in comment.comments():
            get_comments(more_comments, comments_list)

In [19]:
repred

["Whatever happens, we must not become the left. If we do lose, let's take the L and regroup in 2022 and 2024.",
 'I’m with you',
 'I’m all for 4 years of litigation and manufactured scandal involving an impeachment.  Eye for an Eye',
 'I’m all for that, too. I’m not for the violence...the right is way above that bullshit.',
 '[removed]',
 'No disrespect but I care very little about the Dutch.',
 "Why? Of course there is a HUGE dissbalance. I've seen a lot of your nation and culture, you've (i assume) seen little to none of mine.",
 'What a silly comment. You care so little you wrote a comment about your lack of care? Hehe. Ozzie observer here, I love the Dutch! Tot ziens! And to be fair, the comment is accurate, both Biden and Harris would comfortably sit on the right of the chamber in most countries these days',
 "What is it about the left you despise?  Like in my country I'm considered a rightist but in the US I'd be a commie liberal scum",
 ">What is it about the left you despise? 

In [20]:
#count number of responses
num_repred = len(repred)

print(num_repred)

2479


In [21]:
#check type of data in the lists
type(repred[0])

str

In [22]:
#remove all emojis
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

repred1 = []
for string in repred:
    repredem = emoji_pattern.sub('', string)
    repred1.append(repredem)

In [23]:
# remove any deleted or removed comments
for i, j in enumerate(repred1):
    if j == '''[removed]''' or j == '''[deleted]''':
        repred1.pop(i)

In [24]:
len(repred1)

2229

In [25]:
#tokenize
repred2 = []
for i in repred1:
    words = word_tokenize(i)
    repred2.append(words)


In [26]:
# remove all the punctuation
repred3 = repred2

for i in repred3:
    for index_j, j in enumerate(i):
        if j in punct:
            i.pop(index_j)

In [27]:
#lemmatize
repred4 = []

for i in repred3: 
    repred4.append([lemmatizer.lemmatize(j) for j in i])

In [28]:
#lower all cases
repred5 = [[j.lower() for j in i] for i in repred4]

In [29]:
#export to dataframe
big_ef = pd.DataFrame()

big_ef['comments'] = repred1
big_ef['tokenized'] = repred2
big_ef['lemmatized'] = repred4
big_ef['lower'] = repred5

In [30]:
big_ef

,comments,tokenized,lemmatized,lower
0,"Whatever happens, we must not become the left....","[Whatever, happens, we, must, not, become, the...","[Whatever, happens, we, must, not, become, the...","[whatever, happens, we, must, not, become, the..."
1,I’m with you,"[I, ’, m, with, you]","[I, ’, m, with, you]","[i, ’, m, with, you]"
2,I’m all for 4 years of litigation and manufact...,"[I, ’, m, all, for, 4, years, of, litigation, ...","[I, ’, m, all, for, 4, year, of, litigation, a...","[i, ’, m, all, for, 4, year, of, litigation, a..."
3,"I’m all for that, too. I’m not for the violenc...","[I, ’, m, all, for, that, too, I, ’, m, not, f...","[I, ’, m, all, for, that, too, I, ’, m, not, f...","[i, ’, m, all, for, that, too, i, ’, m, not, f..."
4,No disrespect but I care very little about the...,"[No, disrespect, but, I, care, very, little, a...","[No, disrespect, but, I, care, very, little, a...","[no, disrespect, but, i, care, very, little, a..."
...,...,...,...,...
2224,Well thanks for explaining. As non American i...,"[Well, thanks, for, explaining, As, non, Ameri...","[Well, thanks, for, explaining, As, non, Ameri...","[well, thanks, for, explaining, as, non, ameri..."
2225,"Differs by state and by county, so it’s all ov...","[Differs, by, state, and, by, county, so, it, ...","[Differs, by, state, and, by, county, so, it, ...","[differs, by, state, and, by, county, so, it, ..."
2226,Depends on how the state wants to report it. T...,"[Depends, on, how, the, state, wants, to, repo...","[Depends, on, how, the, state, want, to, repor...","[depends, on, how, the, state, want, to, repor..."
2227,How are taxes socialist? By your definition so...,"[How, are, taxes, socialist, By, your, definit...","[How, are, tax, socialist, By, your, definitio...","[how, are, tax, socialist, by, your, definitio..."


In [31]:
#export to csv
big_df.to_excel('repreddit.xlsx')

# Joe Biden Reddit

In [32]:
# reddit post 3
# post ID
post = reddit.submission(id="jpu92e")
                
#empty List
joered = []

# all comments and replies into empty list
for comment in post.comments:
    if len(joered) < 3000:  # set a limit here
        get_comments(comment, joered)

# collect data
def get_comments(comment, comments_list):
    
    if isinstance(comment, praw.models.Comment):
        text = comment.body.replace("\n", " ")
        comments_list.append(text)

        for reply in comment.replies:
            get_comments(reply, comments_list)
    
    elif isinstance(comment, praw.models.MoreComments):
        for more_comments in comment.comments():
            get_comments(more_comments, comments_list)

In [33]:
joered

['FREEE AT LAST   FREE AT LASTTTT  THANK GOD ALMIGHTY WERE FREE AT LASTTT',
 "Aaaah! After 4 thousand years we're free! It's time to lead the earth!  Oh yes! Joe got elected! Prepare Senators and House members with attitude!",
 'I understood that reference.',
 'As did I',
 'We are not free or safe.  This victory was razor thin.  2022 is coming at us like a freight train.',
 'While I highly encourage everyone to savor this moment and let out a sigh of relief, know that this fight is not over. Georgia will be having 2 run off elections and this is incredibly important if we want to give our President a fighting chance at getting anything done in the Senate.  If you can afford it, please consider donating to either Jon Ossoff [here](https://electjon.com/), Raphael Warnock [here](https://warnockforgeorgia.com/), or the amazing Fair Fight organization headed by Stacey Abrams [here](https://fairfight.com/).   If you have the time, please consider phone banking for either of these candidates;

In [34]:
#count number of responses
num_joered = len(joered)

print(num_joered)

1217


In [35]:
#check type of data in the lists
type(joered[0])

str

In [36]:
#remove all emojis
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

joered1 = []
for string in joered:
    joeredem = emoji_pattern.sub('', string)
    joered1.append(joeredem)

In [37]:
# remove any deleted or removed comments
for i, j in enumerate(joered1):
    if j == '''[removed]''' or j == '''[deleted]''':
        joered1.pop(i)

In [38]:
len(joered1)

1157

In [39]:
#tokenize
joered2 = []
for i in joered1:
    words = word_tokenize(i)
    joered2.append(words)

In [40]:
# remove all the punctuation
joered3 = joered2

for i in joered3:
    for index_j, j in enumerate(i):
        if j in punct:
            i.pop(index_j)

In [41]:
#lemmatize
joered4 = []

for i in joered3: 
    joered4.append([lemmatizer.lemmatize(j) for j in i])

In [42]:
#lower all cases
joered5 = [[j.lower() for j in i] for i in joered4]

In [43]:
#export to dataframe
big_ff = pd.DataFrame()

big_ff['comments'] = joered1
big_ff['tokenized'] = joered2
big_ff['lemmatized'] = joered4
big_ff['lower'] = joered5

In [44]:
big_ff

,comments,tokenized,lemmatized,lower
0,FREEE AT LAST FREE AT LASTTTT THANK GOD ALM...,"[FREEE, AT, LAST, FREE, AT, LASTTTT, THANK, GO...","[FREEE, AT, LAST, FREE, AT, LASTTTT, THANK, GO...","[freee, at, last, free, at, lastttt, thank, go..."
1,Aaaah! After 4 thousand years we're free! It's...,"[Aaaah, After, 4, thousand, years, we, 're, fr...","[Aaaah, After, 4, thousand, year, we, 're, fre...","[aaaah, after, 4, thousand, year, we, 're, fre..."
2,I understood that reference.,"[I, understood, that, reference]","[I, understood, that, reference]","[i, understood, that, reference]"
3,As did I,"[As, did, I]","[As, did, I]","[as, did, i]"
4,We are not free or safe. This victory was raz...,"[We, are, not, free, or, safe, This, victory, ...","[We, are, not, free, or, safe, This, victory, ...","[we, are, not, free, or, safe, this, victory, ..."
...,...,...,...,...
1152,Trump reddit right now be like https://youtu....,"[Trump, reddit, right, now, be, like, https, /...","[Trump, reddit, right, now, be, like, http, //...","[trump, reddit, right, now, be, like, http, //..."
1153,I think it’s deleted lmao,"[I, think, it, ’, s, deleted, lmao]","[I, think, it, ’, s, deleted, lmao]","[i, think, it, ’, s, deleted, lmao]"
1154,Nah I was in there a minute ago,"[Nah, I, was, in, there, a, minute, ago]","[Nah, I, wa, in, there, a, minute, ago]","[nah, i, wa, in, there, a, minute, ago]"
1155,[Hey Donnie](https://youtu.be/1N-4blW_8ic),"[Hey, Donnie, (, https, //youtu.be/1N-4blW_8ic]","[Hey, Donnie, (, http, //youtu.be/1N-4blW_8ic]","[hey, donnie, (, http, //youtu.be/1n-4blw_8ic]"


In [45]:
#export to csv
big_ff.to_excel('joereddit.xlsx')